### Užduotis
Sukurti Python pagrindu veikiančią programą, kuri:

- Nuskaitytų pateiktą vaizdo įrašą ir suskirstytų jį į atskirus kadrus.    
- Analizuotų kiekvieno kadro kokybę, įvertindama sugadinimo lygį dėl kameros judėjimo (pvz., suliejimą, iškraipymą, staigius pokyčius).   
- Naudodama klasterizavimo metodus (pvz., K-Means, DBSCAN, hierarchinį klasterizavimą), suskirstytų kadrus į grupes pagal sugadinimo laipsnį.   
- Identifikuotų labiausiai sugadintus kadrus ir vizualiai juos pažymėtų analizės ataskaitoje arba sugeneruotame vaizdo įraše.   

In [12]:
%pip install opencv-python
%pip install matplotlib
%pip install scikit-learn

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Extract frames from video
def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return
    
    frame_count = 0
    os.makedirs(output_folder, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames.")

# Calculate blur score using Laplacian variance
def calculate_blur(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# Analyze frames for motion blur
def analyze_frames(frame_folder):
    blur_scores = []
    frame_paths = sorted(os.listdir(frame_folder))

    for frame_name in frame_paths:
        frame_path = os.path.join(frame_folder, frame_name)
        image = cv2.imread(frame_path)
        
        if image is None:
            continue

        blur_value = calculate_blur(image)
        blur_scores.append((frame_name, blur_value))

    return blur_scores

# Cluster frames based on blur score using K-Means
def cluster_frames(blur_scores, num_clusters=3):
    blur_values = np.array([score[1] for score in blur_scores]).reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(blur_values)

    clustered_frames = {i: [] for i in range(num_clusters)}
    for i, (frame_name, blur_value) in enumerate(blur_scores):
        clustered_frames[labels[i]].append((frame_name, blur_value))

    return clustered_frames

# Identify most distorted frames
def identify_blurred_frames(clustered_frames):
    most_blurred_cluster = min(clustered_frames, key=lambda k: np.mean([b[1] for b in clustered_frames[k]]))
    return clustered_frames[most_blurred_cluster]

# Save results and visualize
def save_analysis(blur_scores, clustered_frames, output_folder):
    plt.figure(figsize=(10, 5))

    blur_values = [b[1] for b in blur_scores]
    plt.hist(blur_values, bins=30, color='blue', alpha=0.7)
    plt.xlabel("Blur Score")
    plt.ylabel("Number of Frames")
    plt.title("Blur Distribution Across Frames")
    
    plt.savefig(os.path.join(output_folder, "blur_analysis.png"))
    plt.close()
    
    with open(os.path.join(output_folder, "analysis_report.txt"), "w") as f:
        f.write("Frame Analysis Report\n")
        f.write("====================\n\n")
        for cluster, frames in clustered_frames.items():
            f.write(f"Cluster {cluster} (Avg Blur Score: {np.mean([b[1] for b in frames]):.2f})\n")
            for frame_name, blur_value in frames[:10]:  # Show first 10 frames per cluster
                f.write(f"  - {frame_name}: Blur Score {blur_value:.2f}\n")
            f.write("\n")

    print(f"Analysis saved to {output_folder}/analysis_report.txt and blur histogram.")

# Main function
def process_video(video_path, output_folder="output_frames"):
    extract_frames(video_path, output_folder)
    blur_scores = analyze_frames(output_folder)
    clustered_frames = cluster_frames(blur_scores)
    blurred_frames = identify_blurred_frames(clustered_frames)
    save_analysis(blur_scores, clustered_frames, output_folder)

    return clustered_frames, blurred_frames

# Run the process
video_file = "videoplayback.mp4"
process_video(video_file)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Extracted 760 frames.
Analysis saved to output_frames/analysis_report.txt and blur histogram.


({0: [('frame_0001.jpg', np.float64(436.65807268036286)),
   ('frame_0002.jpg', np.float64(403.27844183347736)),
   ('frame_0003.jpg', np.float64(390.9241883671325)),
   ('frame_0004.jpg', np.float64(384.2043656677246)),
   ('frame_0005.jpg', np.float64(377.5126730538074)),
   ('frame_0012.jpg', np.float64(484.19082865982995)),
   ('frame_0013.jpg', np.float64(469.8755338473662)),
   ('frame_0014.jpg', np.float64(459.6934087082851)),
   ('frame_0015.jpg', np.float64(448.30104451073555)),
   ('frame_0016.jpg', np.float64(453.6005215250462)),
   ('frame_0017.jpg', np.float64(438.2805398400577)),
   ('frame_0018.jpg', np.float64(422.95770335013304)),
   ('frame_0019.jpg', np.float64(408.4536836909353)),
   ('frame_0020.jpg', np.float64(403.4296013719346)),
   ('frame_0021.jpg', np.float64(399.62911916202495)),
   ('frame_0022.jpg', np.float64(407.9460967603895)),
   ('frame_0023.jpg', np.float64(404.19631047423564)),
   ('frame_0024.jpg', np.float64(415.8786765163846)),
   ('frame_0025.jp

### Adding video where blur is highlighted

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Extract frames from video
def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    frame_count = 0
    os.makedirs(output_folder, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames.")

# Calculate blur score using Laplacian variance
def calculate_blur(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# Analyze frames for motion blur
def analyze_frames(frame_folder):
    blur_scores = []
    frame_paths = sorted(os.listdir(frame_folder))

    for frame_name in frame_paths:
        frame_path = os.path.join(frame_folder, frame_name)
        image = cv2.imread(frame_path)

        if image is None:
            continue

        blur_value = calculate_blur(image)
        blur_scores.append((frame_name, blur_value))

    return blur_scores

# Cluster frames based on blur score using K-Means
def cluster_frames(blur_scores, num_clusters=3):
    blur_values = np.array([score[1] for score in blur_scores]).reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(blur_values)

    clustered_frames = {i: [] for i in range(num_clusters)}
    for i, (frame_name, blur_value) in enumerate(blur_scores):
        clustered_frames[labels[i]].append((frame_name, blur_value))

    return clustered_frames

# Identify most distorted frames
def identify_blurred_frames(clustered_frames):
    most_blurred_cluster = min(clustered_frames, key=lambda k: np.mean([b[1] for b in clustered_frames[k]]))
    return clustered_frames[most_blurred_cluster]

# Highlight blurred frames and create a new video
def highlight_blurred_frames(frame_folder, blurred_frames, output_video):
    first_frame = cv2.imread(os.path.join(frame_folder, blurred_frames[0][0]))
    height, width, _ = first_frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, 30, (width, height))

    frame_paths = sorted(os.listdir(frame_folder))

    for frame_name in frame_paths:
        frame_path = os.path.join(frame_folder, frame_name)
        frame = cv2.imread(frame_path)

        if frame is None:
            continue

        if frame_name in [b[0] for b in blurred_frames]:
            cv2.rectangle(frame, (10, 10), (width - 10, height - 10), (0, 0, 255), 10)  # Red border

        out.write(frame)

    out.release()
    print(f"New video saved: {output_video}")

# Save results and visualize
def save_analysis(blur_scores, clustered_frames, output_folder):
    plt.figure(figsize=(10, 5))

    blur_values = [b[1] for b in blur_scores]
    plt.hist(blur_values, bins=30, color='blue', alpha=0.7)
    plt.xlabel("Blur Score")
    plt.ylabel("Number of Frames")
    plt.title("Blur Distribution Across Frames")

    plt.savefig(os.path.join(output_folder, "blur_analysis.png"))
    plt.close()

    with open(os.path.join(output_folder, "analysis_report.txt"), "w") as f:
        f.write("Frame Analysis Report\n")
        f.write("====================\n\n")
        for cluster, frames in clustered_frames.items():
            f.write(f"Cluster {cluster} (Avg Blur Score: {np.mean([b[1] for b in frames]):.2f})\n")
            for frame_name, blur_value in frames[:10]:  # Show first 10 frames per cluster
                f.write(f"  - {frame_name}: Blur Score {blur_value:.2f}\n")
            f.write("\n")

    print(f"Analysis saved to {output_folder}/analysis_report.txt and blur histogram.")

# Main function
def process_video(video_path, output_folder="video_output_frames", output_video="highlighted_video.mp4"):
    extract_frames(video_path, output_folder)
    blur_scores = analyze_frames(output_folder)
    clustered_frames = cluster_frames(blur_scores)
    blurred_frames = identify_blurred_frames(clustered_frames)
    save_analysis(blur_scores, clustered_frames, output_folder)
    highlight_blurred_frames(output_folder, blurred_frames, output_video)

    return clustered_frames, blurred_frames

# Run the process
video_file = "videoplayback.mp4"
process_video(video_file)


✅ Extracted 760 frames.
Analysis saved to video_output_frames/analysis_report.txt and blur histogram.
🎥 New video saved: highlighted_video.mp4


({0: [('frame_0027.jpg', np.float64(365.90283684200716)),
   ('frame_0028.jpg', np.float64(375.05012389818364)),
   ('frame_0029.jpg', np.float64(368.79240532722)),
   ('frame_0030.jpg', np.float64(360.7498461223839)),
   ('frame_0031.jpg', np.float64(358.5634088143054)),
   ('frame_0032.jpg', np.float64(366.82528058899305)),
   ('frame_0033.jpg', np.float64(358.69257653235917)),
   ('frame_0034.jpg', np.float64(361.5600267921542)),
   ('frame_0035.jpg', np.float64(358.1707189941218)),
   ('frame_0036.jpg', np.float64(363.98045153113463)),
   ('frame_0037.jpg', np.float64(363.47981919485966)),
   ('frame_0038.jpg', np.float64(364.32227821150417)),
   ('frame_0039.jpg', np.float64(363.8664352462015)),
   ('frame_0040.jpg', np.float64(369.58224317760704)),
   ('frame_0041.jpg', np.float64(365.6837462528182)),
   ('frame_0042.jpg', np.float64(365.07733652974815)),
   ('frame_0043.jpg', np.float64(361.5926626753441)),
   ('frame_0044.jpg', np.float64(366.00039968050737)),
   ('frame_0045.j